In [49]:
pip install ucimlrepo #installed ucimlrepo using pip command to import the package ucimlrepo

# 1. Belows given codes with some already done basic pre-processing

In [50]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

X_initial_shape = print(X.shape)                                                 #added to know the shape of dataframes(X and y ) before dropping
y_initial_shape = print(y.shape)


print(X.columns)                                                                 #to know the features and target columns
print(y.columns)

X = X.drop(['fnlwgt', 'education-num'],axis=1)                                   #dropped these two columns

y = y[~X.isna().any(axis=1)]
X = X[~X.isna().any(axis=1)]

X_shape_after_dropping = print(X.shape)                                          #added to know the shape of dataframes(X and y) after dropping
y_shape_after_dropping = print(y.shape)

columns = list(X.columns)
for col in columns:
	col_list = list(X[col])
	if isinstance(col_list[0], int):
		print('Column name: `{}\', type: integer'.format(col))
		print('')
	else:
		print('Column name: `{}\', type: categorical'.format(col))
		print('\tPossible values: {}'.format(list(set(col_list))))
		print('')


(48842, 14)
(48842, 1)
Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')
Index(['income'], dtype='object')
(47621, 12)
(47621, 1)
Column name: `age', type: integer

Column name: `workclass', type: categorical
	Possible values: ['Never-worked', 'Self-emp-inc', '?', 'Local-gov', 'Without-pay', 'Federal-gov', 'Private', 'State-gov', 'Self-emp-not-inc']

Column name: `education', type: categorical
	Possible values: ['Assoc-voc', 'Prof-school', 'Some-college', 'Assoc-acdm', '5th-6th', '10th', 'Preschool', '12th', '11th', 'Doctorate', '7th-8th', '1st-4th', '9th', 'HS-grad', 'Bachelors', 'Masters']

Column name: `marital-status', type: categorical
	Possible values: ['Divorced', 'Married-civ-spouse', 'Married-spouse-absent', 'Separated', 'Widowed', 'Never-married', 'Married-AF-spouse']

Column name: `occupation'

# Observations:
- Dataset has been divided into X and y dataframes
- The initial shape is :(48842, 14) , (48842, 1) respectively.

- After dropping fnlwgt, education-num , we are left with (47621, 12),
(47621, 1)

- **to be noted that Few columsn are still not cleaned and contains dirty data such as '?'**


# More Data Cleaning
-Decided to do more Data Cleaning as columns "workclass","occupation"and "native country" have "?" .

In [51]:
missing_values_X = X.isna().sum()                                       #trying to check null values in X and y dataframes
print(missing_values_X)

missing_values_y = y.isna().sum()
print("Missing values in Traget column",missing_values_y)

age               0
workclass         0
education         0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64
Missing values in Traget column income    0
dtype: int64


In [52]:
X.replace('?',np.NaN,inplace=True) # we don't have missing values in dataframes but we do have ? in few columns so , i am replacing with "nan " values

In [53]:
y = y[~X.isna().any(axis=1)] # after replacing '?' with 'nan' values , i am not taking those in my X and y dataframes
X = X[~X.isna().any(axis=1)]

In [54]:
# Again Checking for missing values for double surity
missing_values_X = X.isna().sum()
print("Missing values in each column of X after handling '?':\n", missing_values_X)
missing_values_y = y.isna().sum()
print(missing_values_y)

Missing values in each column of X after handling '?':
 age               0
workclass         0
education         0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64
income    0
dtype: int64


In [55]:
X_shape = print(X.shape)
y_shape = print(y.shape)

(45222, 12)
(45222, 1)


Observation
- Therefore , this leaves us with information of about 45222 people.:

# Task 1.1
***Strategize [15 pts] For each of the twelve pieces of demographic information, explain in 1-2 sentences how you will represent that piece of information as one or more real numbers and why. Note: there is not a single “right” way to do this!***


# Answers :
1. Age: We are going to represent as the integer value only because age is a continuous variable  can be used directly in its numerical form.

2. Workclass: We are going to use one-hot encoding to represent each category as a binary feature. As this ensures no ordinal relationship is implied between categories.

3. Education: Decided to apply one-hot encoding to each education level. Since education is a nominal categorical variable, this avoids implying any order among levels.

4. Marital-Status: Going to use one-hot encoding for each marital status category because this takes each status as a distinct category without suggesting any hierarchy.

5. Occupation: We will apply one-hot encoding to convert occupation into binary features. Each job role is distinct, and this method prevents false assumptions about relationships between them.

6. Relationship: We will be using one-hot encoding to handle relationship categories. This will represent each type of relationship as an independent feature, avoiding any implied ranking.

7. Race: Will apply one-hot encoding to represent each race category. This ensures that different races are treated independently without any ordinal assumptions.

8. Sex: We will go with binary encoding (0 for Female, 1 for Male). With only two categories, binary encoding is efficient and sufficient.

9. Capital-Gain: Will keep the integer value as it because Capital-gain is already a numeric feature and can be used directly.

10. Capital-Loss: We will use the integer value as is. This variable is already numerical and requires no transformation.

11. Hours-Per-Week: Use the raw integer value, as hours worked per week is a continuous variable.

12. Native-Country: Will apply one-hot encoding to represent each country as a distinct binary feature. This avoids implying any geographical or economic ranking between countries.








# Task 1.2 Implement [15 pts]
Write code to implement the featurization strategy you just described. How many features do you have in total for each data instance?
Note: I recommend including, at this stage, a feature that is always equal to 1 to account for the intercept term in the linear model we will train in the next problem.

In [56]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler


categorical_cols = X.select_dtypes(include=['object']).columns.tolist()            # taking all the columns having datatype object from X and keeping it in  Categorical Cols
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()    # taking all the columns having datatype int or float from X and keeping it in  Numerical Cols

for col in categorical_cols:
    unique_categories = X[col].nunique()
    print(f"{col}: {unique_categories} unique categories ")

print("x" * 100)

# Featurization for categorical
encoder = OneHotEncoder(sparse_output=False, drop='first')                         # One-Hot Encoding for Categorical features
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_cols]))

# Featurization for numerical
X_numerical = X[numerical_cols].reset_index(drop=True)                             # Selected the numerical columns from X dataframe and resetting the index to avoid index misalignment
X_featurized = pd.concat([X_numerical, X_encoded.reset_index(drop=True)], axis=1)  # Concatenated the numerical features and the one-hot encoded categorical features into a single DataFrame

scaler = StandardScaler()                                                          # Applyed standardization on the numerical features in X_featurized to normalize them
X_featurized[numerical_cols] = scaler.fit_transform(X_featurized[numerical_cols])

#Added Intercept Term
X_featurized['intercept'] = 1                                                      # Adding a column of ones to X_featurized dataframe to serve as an intercept term for linear models




print("Shape of the featurized data:", X_featurized.shape)
print("First few rows of the featurized data:\n", X_featurized.head())



total_features = X_featurized.shape[1]                                              # Total number of features
print("Total number of features after transformation:", total_features)


workclass: 7 unique categories 
education: 16 unique categories 
marital-status: 7 unique categories 
occupation: 14 unique categories 
relationship: 6 unique categories 
race: 5 unique categories 
sex: 2 unique categories 
native-country: 41 unique categories 
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Shape of the featurized data: (45222, 95)
First few rows of the featurized data:
         age  capital-gain  capital-loss  hours-per-week    0    1    2    3  \
0  0.034201      0.142888      -0.21878       -0.078120  0.0  0.0  0.0  0.0   
1  0.866417     -0.146733      -0.21878       -2.326738  0.0  0.0  0.0  1.0   
2 -0.041455     -0.146733      -0.21878       -0.078120  0.0  1.0  0.0  0.0   
3  1.093385     -0.146733      -0.21878       -0.078120  0.0  1.0  0.0  0.0   
4 -0.798015     -0.146733      -0.21878       -0.078120  0.0  1.0  0.0  0.0   

     4    5  ...   81   82   83   84   85   86   87   88   89  intercept  
0  1.

Explanation:
- Numerical Features: The count remains the same after normalization.
- One-Hot Encoded Categorical Features: For each categorical column, the number of features is (number of unique categories - 1) because of drop='first'.
- Intercept Term: Adds 1 to the total count.

# Trying to understand y dataframe more
- since our target column is income , so we have checked for any kind of >,<,=,whitespaces
- our goal is to predict yes/no whether
their yearly income > $50,000
- so for that we have to map income column into two parts (<=50k = -1 and >50k =1)
- we need to check whether it's an imbalanced dataset or balanced before procedding ahead to get rid of any kind of baised behaviour.

In [57]:
if isinstance(y, pd.DataFrame):
    y = y['income']


y = y.str.strip().str.replace('.', '', regex=False)                             # checked all extra whitespace or punctuation


y_numeric = y.replace({'<=50K': -1, '>50K': 1})                                 # We are replacing the string labels ('<=50K' and '>50K') with numeric values -1 and 1


print("Class distribution after converting labels:")
print(y_numeric.value_counts(normalize=True))                                   # this vaerifies the conversion of class labels into -1 and 1


Class distribution after converting labels:
income
-1    0.752156
 1    0.247844
Name: proportion, dtype: float64


<ipython-input-57-2186d00422b2>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_numeric = y.replace({'<=50K': -1, '>50K': 1})                                 # We are replacing the string labels ('<=50K' and '>50K') with numeric values -1 and 1


# Observations :
1. Here, <=50K -> -1 : Approximately 75.2% of the instances in the dataset have an income less than or equal to $50K.

2. and, >50K -> 1 : Approximately 24.8% of the instances have an income greater than $50K.

3. This confirms that this dataset is imbalanced, with more people earning <=50K than >50K.

4. Hence , we need to handle this imbalance to prevent it from biased predictions favoring the majority class.



# Task 2 Classify! [60 pts]
Now that you have featurized the data, we will try to classify it using logistic regression.
Split the data into training, validation, and testing data (roughly in 80% / 10% / 10% of the data
in each).

In [58]:
from sklearn.model_selection import train_test_split


X_train, X_temp, y_train, y_temp = train_test_split(X_featurized, y_numeric, test_size=0.2,
                                                    random_state=42, stratify=y_numeric) # Splitted the data into training (80%) and temporary (20%) sets


X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5,
                                                random_state=42, stratify=y_temp         # Splitted the temporary set into validation (10%) and testing (10%) sets
)


print("Training set size:", X_train.shape[0])                                            # to know the sizes of each sets train, test and validation set
print("Validation set size:", X_val.shape[0])
print("Test set size:", X_test.shape[0])


print("\nClass distribution in Training set:")                                          # to verify class distribution in each split
print(y_train.value_counts(normalize=True))
print("\nClass distribution in Validation set:")
print(y_val.value_counts(normalize=True))
print("\nClass distribution in Test set:")
print(y_test.value_counts(normalize=True))


Training set size: 36177
Validation set size: 4522
Test set size: 4523

Class distribution in Training set:
income
-1    0.752163
 1    0.247837
Name: proportion, dtype: float64

Class distribution in Validation set:
income
-1    0.752101
 1    0.247899
Name: proportion, dtype: float64

Class distribution in Test set:
income
-1    0.752156
 1    0.247844
Name: proportion, dtype: float64


# Handling Class Imbalance with Class Weights


In [59]:
import numpy as np

# Calculate class weights based on training set
classes = np.unique(y_train)                                                     # This gives you an array of all distinct classes present in the y_training data only(because model learns from training set)
class_counts = y_train.value_counts().sort_index()                               # gives the count of each class labels having unique values
total_samples = len(y_train)

class_weights = {cls: total_samples / (len(classes) * count) for cls, count in class_counts.items()}

print("\nClass Weights:")
print(class_weights)



Class Weights:
{-1: 0.6647495498144133, 1: 2.0174548293553425}


# Observations
1. `Class -1 with weight:0.6647`

- This weight indicates that informations of this class are less critical for the model's learning process compared to the instances of class 1 .

- Since this class has more examples in the dataset, the weight is less than which means that the model should not focus excessively on this class when making predictions.

2. `Class 1 with weight 2.0174`

- This weight is significantly greater than 1 suggesting that instances of this class are more critical for the model's learning.

- Since this class has lower count compared to class -1 , assigning a higher weight helps the model pay more attention to correctly classifying these instances.

# Task 2.2 SGD [20 pts]
Write code to implement stochastic gradient descent for minimizing the logistic loss.

In [60]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))                                                  # returns sigmoid of z

def compute_gradient(X_i, y_i, w, lambda_reg, class_weights):         # Parameters:  X_i: Feature vector for sample i  , y_i: Label for sample i , w: Current weight vector, lambda_reg: Regularization parameter, class_weights: Dictionary mapping class labels to weights




    z = y_i * np.dot(w, X_i)                                                     # Calculated  the linear combination of features and weights y_i * (w^T * X_i)
    prediction = sigmoid(z)
    gradient = -y_i * X_i * (1 - prediction)                                     # Calculated the gradient of the loss with respect to the weights

                                                                                 # Applied class weights
    weight = class_weights[y_i]
    gradient *= weight                                                           # Scaled the gradient by the class weight


    gradient += lambda_reg * w                                                   # L2 Regualarization term

    return gradient                                                              # Returns Gradient vector






In [61]:
# Computing the regularized logistic loss across all samples, taking into account the class weights.

def compute_loss(X, y, w, lambda_reg, class_weights):
    """
    Parameters:
    - X: Feature matrix (NumPy array)
    - y: Labels (NumPy array, values -1 or 1)
    - w: Weight vector (NumPy array)
    - lambda_reg: Regularization parameter (float)
    - class_weights: Dictionary mapping class labels to weights (dict)
    """


    z = y * np.dot(X, w)                    #Calculating the linear combination of inputs and weights: z = y * (X * w)


    loss_terms = np.log(1 + np.exp(-z))    # Computing the loss terms using the logistic loss formula: log(1 + exp(-z))


    weighted_loss = 0                      # this is to hold the total weighted loss
    for cls in class_weights:
        cls_mask = (y == cls)
        weighted_loss += np.sum(class_weights[cls] * loss_terms[cls_mask])


    loss = weighted_loss / len(y)           # Average the total weighted loss


    loss += (lambda_reg / 2) * np.sum(w ** 2)  # Add regularization

    return loss


In [62]:
def stochastic_gradient_descent(X, y, w_init, lambda_reg, learning_rate, num_iterations, class_weights):

    w = w_init.copy()                                                            # a copy of the initial weights to avoid modifying the original
    m, n = X.shape                                                               # Get the number of samples (m) and features (n) from the feature matrix


    for epoch in range(num_iterations):                                           # Loop through the specified number of iterations (epochs)
        for i in range(m):
            idx = np.random.randint(m)                                            # here update the model weights using few randomly selected samples m at each iteration.
            X_i = X[idx]
            y_i = y[idx]

            gradient = compute_gradient(X_i, y_i, w, lambda_reg, class_weights)   # calling Compute gradient function to calculate gradient


            w -= learning_rate * gradient                                         # Update weights using gradient and learning rate


        if (epoch + 1) % 100 == 0:                                                # Print loss every 100 epochs to moniter training loss
            loss = compute_loss(X, y, w, lambda_reg, class_weights)
            print(f"Epoch {epoch+1}/{num_iterations}, Loss: {loss:.4f}")

    return w                                                                      # RETURN Trained weight vector


# 2.3 Accuracy [10 pts]
Recall that the logistic regression classifier outputs the probability that the correct label for an input
is +1. Let’s say that the classifier is “correct” if the output it assigns to the correct label is at least
50%, and the train/test “accuracy” is the percentage of the train/test set that the classifier gets
correct.
Write code to calculate the train and test set accuracy

In [63]:
def predict(X, w):           # to Make predictions using the logistic regression model

    probabilities = sigmoid(np.dot(X, w))
    return np.where(probabilities >= 0.5, 1, -1)

def calculate_accuracy(X, y, w):   # to Calculate the accuracy of the model

    predictions = predict(X, w)
    accuracy = np.mean(predictions == y)
    return accuracy


# TASK 2.4 Train! [25 pts]
- Use your SGD code to train a logistic regression classifier.
- Notice that there are three “hyperparameters”—the regularization parameter, λ, the stepsize, η, and the number of iterations of SGD (and there may actually be a fourth, the minibatch size, b, if
you choose to do that)—that you need to choose.
- You should make some amount of effort to choose these hyperparameters in order to get good performance out of your model, but don’t go too crazy.
- Describe how you chose your hyperparameters(guess and check is fine, but describe how you “checked”).
- Report the final test accuracy of the model that you learned. Recall, computing the test accuracy here should be the only time that you touch the test dataset.
- The final test accuracy should be at least 60-70%, otherwise there is probably a bug in your code somewhere! If you’re feeling competitive, it should be possible to get test accuracy in the mid to
high 80% range, but for full points you only need to get 60-70+% accuracy.

In [76]:
# Try 1 : Initialized parameters
w_init = np.zeros(X_train.shape[1])
lambda_reg = 1
learning_rate = 1
num_iterations = 100

# Training the model using SGD
w_final = stochastic_gradient_descent(
    X_train.values,
    y_train.values,
    w_init,
    lambda_reg,
    learning_rate,
    num_iterations,
    class_weights
)

# Evaluating the Model
train_accuracy = calculate_accuracy(X_train.values, y_train.values, w_final)
val_accuracy = calculate_accuracy(X_val.values, y_val.values, w_final)
test_accuracy = calculate_accuracy(X_test.values, y_test.values, w_final)

print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 100/100, Loss: 0.8143

Training Accuracy: 75.21%
Validation Accuracy: 75.21%
Test Accuracy: 75.24%


In [72]:
# Try 2 : Initialized parameters
w_init = np.zeros(X_train.shape[1])
lambda_reg = 0.01
learning_rate = 0.01
num_iterations = 100

# Training the model using SGD
w_final = stochastic_gradient_descent(
    X_train.values,
    y_train.values,
    w_init,
    lambda_reg,
    learning_rate,
    num_iterations,
    class_weights
)

# Evaluating the Model
train_accuracy = calculate_accuracy(X_train.values, y_train.values, w_final)
val_accuracy = calculate_accuracy(X_val.values, y_val.values, w_final)
test_accuracy = calculate_accuracy(X_test.values, y_test.values, w_final)

print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 100/100, Loss: 0.4538

Training Accuracy: 77.37%
Validation Accuracy: 77.71%
Test Accuracy: 77.18%


In [73]:
# Try 3 : Initialized parameters
w_init = np.zeros(X_train.shape[1])
lambda_reg = 0.01
learning_rate = 0.001
num_iterations = 500

# Training the model using SGD
w_final = stochastic_gradient_descent(
    X_train.values,
    y_train.values,
    w_init,
    lambda_reg,
    learning_rate,
    num_iterations,
    class_weights
)
# Evaluating the Model
train_accuracy = calculate_accuracy(X_train.values, y_train.values, w_final)
val_accuracy = calculate_accuracy(X_val.values, y_val.values, w_final)
test_accuracy = calculate_accuracy(X_test.values, y_test.values, w_final)

print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 100/500, Loss: 0.4497
Epoch 200/500, Loss: 0.4501
Epoch 300/500, Loss: 0.4504
Epoch 400/500, Loss: 0.4495
Epoch 500/500, Loss: 0.4498

Training Accuracy: 79.46%
Validation Accuracy: 79.72%
Test Accuracy: 79.31%


In [75]:
# Try 4 : Initialized parameters
w_init = np.zeros(X_train.shape[1])
lambda_reg = 0.001  # Lower regularization
learning_rate = 0.01  # Increased learning rate
num_iterations = 500  # Keep the same

# Training the model using SGD
w_final = stochastic_gradient_descent(
    X_train.values,
    y_train.values,
    w_init,
    lambda_reg,
    learning_rate,
    num_iterations,
    class_weights
)

# Evaluating the Model
train_accuracy = calculate_accuracy(X_train.values, y_train.values, w_final)
val_accuracy = calculate_accuracy(X_val.values, y_val.values, w_final)
test_accuracy = calculate_accuracy(X_test.values, y_test.values, w_final)

print(f"\nTraining Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 100/500, Loss: 0.4036
Epoch 200/500, Loss: 0.4024
Epoch 300/500, Loss: 0.4112
Epoch 400/500, Loss: 0.4007
Epoch 500/500, Loss: 0.4019

Training Accuracy: 80.61%
Validation Accuracy: 80.94%
Test Accuracy: 80.17%


- I chose a "guess and check" approach by adjusting one hyperparameter at a time which allowed me to isolate the effects of each change. After each adjustment, I evaluated the model's performance using accuracy metrics and loss values.


- For the initial run (Try 1), I set the regularization parameter to a high value (1) and the learning rate to 1, which is generally not recommended but served as a baseline for assessing the model's performance.

- I reduced the learning rate from 1 to 0.01 in Try 2 to allow the model to converge more effectively. As we can see , loss decreased in try2 to Loss: 0.4538.

- I experimented with the number of iterations to see how increased training time affected accuracy. I moved from 100 to 500 in subsequent tries to observe whether more training epochs improved the model's ability to learn.
